定义多输入通道互相关操作

In [14]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

定义多输入输出通道互相关操作

In [15]:
def corr2d_multi_in_out(X, K):
    # stack 沿 dim = 0，即最外层插入一个新的维度
    # 堆叠 c_0 个二维张量
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

使用全连接层实现 1 * 1 卷积

In [20]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
# assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
print(Y1)
print(Y2)

tensor([[[-1.3019, -1.7904,  1.8077],
         [-0.1874,  3.9721,  2.7824],
         [ 0.6703,  0.0835,  0.0187]],

        [[ 0.0804,  0.6341,  2.5882],
         [-0.8471,  2.3077,  1.7507],
         [ 0.4265,  1.2192, -0.9318]]])
tensor([[[-1.3019, -1.7904,  1.8077],
         [-0.1874,  3.9721,  2.7824],
         [ 0.6703,  0.0835,  0.0187]],

        [[ 0.0804,  0.6341,  2.5882],
         [-0.8471,  2.3077,  1.7507],
         [ 0.4265,  1.2192, -0.9318]]])
